<img style="float: left;;" src='Figures/alinco.png' /></a>

# Modulo I: Naive Bayes para Análisis de Sentimientos


Aplicaremos Naive Bayes para el análisis de sentimientos de tweets, lo que haremos en este notebook será:

* Entrenar un modelo de Naive Bayes para el análisis de sentimientos
* Probaremos el modelo creado
* Calcularemos las proporciones de palabras positivas a palabras negativas
* Veremos una función de análisis de errores
* Predeciremos un tweet


In [1]:
from utils import process_tweet, lookup
import pdb
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer
from os import getcwd

Si aún no se han descargado:
```
nltk.download('stopwords')
nltk.download('twitter_samples')
```

In [2]:
# add folder, tmp2, from our local workspace containing pre-downloaded corpora files to nltk's data path
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [3]:
# get the sets of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

# split the data into two pieces, one for training and one for testing (validation set)
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

# avoid assumptions about the length of all_positive_tweets
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

# Preprocesamiento de los datos

Para cualquier proyecto de aprendizaje automático, una vez que se haya recopilado los datos, el primer paso es el preprocesamiento.

- **Eliminar el ruido**: eliminar las palabras que no dicen mucho sobre el contenido. Estos incluyen todas las palabras comunes como 'I, you, are, is, etc...' que no nos darían suficiente información sobre el sentimiento.
- También eliminaremos los tickers del mercado de valores, los símbolos de retuits, los hipervínculos y los hashtags porque no pueden brindarle mucha información sobre el sentimiento.
- También hay que eliminar toda la puntuación de un tweet. La razón para hacer esto es porque queremos tratar las palabras con o sin puntuación como la misma palabra, en lugar de tratar las palabras "happy", "happy?", "happy!", "happy," and "happy." como palabras diferentes.
- Por último, desea utilizaremos la técnica de stemming para realizar un seguimiento de una sola variación de cada palabra. En otras palabras, trataremos "motivation", "motivated", y "motivate" de manera similar agrupándolos dentro de la misma raíz de "motiv-".

Recuerden que ya tenemos una función para esto (`process_tweet()`)

In [24]:
custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"

# print cleaned tweet
from utilss import Utilities as prep
obj_prep=prep()
print(obj_prep.process_tweet(custom_tweet))

This is the Utilities Constructor
['hello', 'great', 'day', ':)', 'good', 'morn']


## Funciones de ayuda

Para entrenar el modelo, necesitaremos construir un diccionario donde las claves sean una tupla (word, label) y los valores sean la frecuencia correspondiente. Tenga en cuenta que las etiquetas que usaremos aquí son 1 para positivo y 0 para negativo.

Implementaremos la función `lookup()` que toma del diccionario `freqs` una palabra y una etiqueta (1 or 0) y retorna el número de veces que la palabra y esa etiqueta aparecen en la collección de tweets.

Por ejemplo: Dado una lista de sentencias en los tweets: `["i am rather excited", "you are rather happy"]` y la etiqueta 1, la función regresara el diccionario que contiene lo siguiente:

{
    ("rather", 1): 2
    ("happi", 1) : 1
    ("excit", 1) : 1
}

- Observe cómo para cada palabra en la cadena dada, se asigna la misma etiqueta 1 a cada palabra.
- Observe cómo las palabras "i" y "am" no se guardan, ya que fue eliminada por process_tweet porque es una stopword.
- Observe cómo la palabra "rather" aparece dos veces en la lista de tweets, por lo que su valor de recuento es 2.

Crearemos una función `count_tweets()` que toma una lista de tweets como entrada, las limpia, y regresa un diccionario.

- La clave en el diccionario es una tupla que contiene la palabra despues del stemming y su etiqueta de clase, Ej. ("happi", 1).
- El valor del key, será la cantidad de veces que esta palabra aparece en la colección de tweets dada (un número entero).


In [25]:
def count_tweets(result, tweets, ys):
    '''
    Input:
        result: a dictionary that will be used to map each pair to its frequency
        tweets: a list of tweets
        ys: a list corresponding to the sentiment of each tweet (either 0 or 1)
    Output:
        result: a dictionary mapping each pair to its frequency
    '''
    for y, tweet in zip(ys, tweets):
        for word in obj_prep.process_tweet(tweet):
            # define the key, which is the word and label tuple
            pair = (word,y)

            # if the key exists in the dictionary, increment the count
            if pair in result:
                result[pair] += 1

            # else, if the key is new, add it to the dictionary and set the count to 1
            else:
                result[pair] = 1

    return result

In [26]:
# Testing your function


result = {}
tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0, 0, 0, 0]
count_tweets(result, tweets, ys)

{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}

**Expected Output**: {('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}

## Entrenar el modelo usando Naive Bayes

Naive bayes es un algoritmo que se puede usar para aplicaciones como el análisis de sentimientos. Este algoritmo poco tiempo en entrenarse pero tiene un tiempo de predicción corto.


#### ¿Cómo entrenamos un clasificador con Naive Bayes ?

- La primera parte del entrenamiento de un clasificador Naive Bayes es identificar el número de clases que tiene.
- Crearemos una probabilidad para cada clase.


$P(D_{pos})$ denota la probabilidad que el documento sea positivo.
$P(D_{neg})$ denota la probabilidad que el documento sea negativo.

Utilizaremos las fórmulas de la siguiente manera y almacenaremos los valores en un diccionario:

$$P(D_{pos}) = \frac{D_{pos}}{D}\tag{1}$$

$$P(D_{neg}) = \frac{D_{neg}}{D}\tag{2}$$

Donde $ D $ es el número total de documentos, o tweets en este caso, $ D_ {pos} $ es el número total de tweets positivos y $ D_ {neg} $ es el número total de tweets negativos.

#### Prior y Logprior

The prior probability represents the underlying probability in the target population that a tweet is positive versus negative.  In other words, if we had no specific information and blindly picked a tweet out of the population set, what is the probability that it will be positive versus that it will be negative? That is the "prior".

The prior is the ratio of the probabilities $\frac{P(D_{pos})}{P(D_{neg})}$.
We can take the log of the prior to rescale it, and we'll call this the logprior


La probabilidad previa (prior) representa la probabilidad subyacente de la población (tweets) objetivo de que un tweet sea positivo frente a uno negativo. En otras palabras, si no tuviéramos información específica y seleccionáramos a ciegas un tweet del conjunto de la población, ¿cuál es la probabilidad de que sea positivo frente a que sea negativo? Ese es el "previo".

El "prior" es el ratio de las probabilidades $\frac{P(D_{pos})}{P(D_{neg})}$. Podemos tomar el logaritmo del "prior" para cambiar su escala, y lo llamaremos logprior.


$$\text{logprior} = log \left( \frac{P(D_{pos})}{P(D_{neg})} \right) = log \left( \frac{D_{pos}}{D_{neg}} \right)$$.

Note que $log(\frac{A}{B})$ es igual a $log(A) - log(B)$.  Por lo tanto logprior puede calcularse como la diferencia de dos logaritmos (propiedad de logaritmos).


$$\text{logprior} = \log (P(D_{pos})) - \log (P(D_{neg})) = \log (D_{pos}) - \log (D_{neg})\tag{3}$$

#### Probabilidad positiva y negativa de una palabra
Para calcular la probabilidad positiva y la probabilidad negativa de una palabra específica en el vocabulario, usaremos las siguientes entradas:

- $freq_{pos}$ y $freq_{neg}$ son las frecuencias de esa palabra específica en la clase positiva o negativa. En otras palabras, la frecuencia positiva de una palabra es el número de veces que se cuenta la palabra con la etiqueta 1.
- $ N_ {pos} $ y $ N_ {neg} $ son el número total de palabras positivas y negativas para todos los documentos (para todos los tweets), respectivamente.

- $V$ es el número de palabras únicas en todo el conjunto de documentos, para todas las clases, ya sean positivas o negativas.

Los usaremos para calcular la probabilidad positiva y negativa de una palabra específica usando la fórmula:

$$ P(W_{pos}) = \frac{freq_{pos} + 1}{N_{pos} + V}\tag{4} $$
$$ P(W_{neg}) = \frac{freq_{neg} + 1}{N_{neg} + V}\tag{5} $$

Note que añadimos "+1" en el numerador para el smoothing aditivo.  Este [artículo](https://en.wikipedia.org/wiki/Additive_smoothing) explica detalladamente sobre el smoothing aditivo.

#### Log likelihood
Para calcular la loglikelihood (probabilidad mínima) de esa misma palabra, podemos implementar las siguientes ecuaciones:

$$\text{loglikelihood} = \log \left(\frac{P(W_{pos})}{P(W_{neg})} \right)\tag{6}$$

##### Creación del diccionario de  frecuencias (`freqs` )
- Dada la función `count_tweets ()`, podemos calcular un diccionario llamado `freqs` que contiene todas las frecuencias.
- En este diccionario `freqs`, la clave es la tupla (word,label)
- El valor es el número de veces que ha aparecido.



In [7]:
# Build the freqs dictionary for later uses

freqs = count_tweets({}, train_x, train_y)


### Pipeline para la creación de la función  `train_naive_bayes`
Dado un diccionario de frecuencias, `train_x` (una lista de tweets) y un` train_y` (una lista de etiquetas para cada tweet), implementaremos un clasificador Naive Bayes.

##### Calcular $V$
- Podemos calcular el número de palabras únicas que aparecen en el diccionario `freqs` para obtener $ V $ (usando la función` set`).

##### Calcular $freq_{pos}$ y $freq_{neg}$
- Usando el diccionario `freqs`, se puede calcular la frecuencia positiva y negativa de cada palabra $ freq_ {pos} $ y $ freq_ {neg} $.

##### Calcular $N_{pos}$ y $N_{neg}$
- Usando el diccionario `freqs`, también puede calcular el número total de palabras positivas y el número total de palabras negativas $ N_ {pos} $ y $ N_ {neg} $.

##### Calcular $D$, $D_{pos}$, $D_{neg}$
- Usando la lista de etiquetas de entrada `train_y`, calcularemos el número de documentos (tweets) $ D $, así como el número de documentos positivos (tweets) $ D_ {pos} $ y el número de documentos negativos (tweets) $ D_ { neg} $.
- Calcularemos la probabilidad de que un documento (tweet) sea positivo $ P (D_ {pos}) $, y la probabilidad de que un documento (tweet) sea negativo $ P (D_ {neg}) $

##### Calcularemos el logprior
-  $log(D_{pos}) - log(D_{neg})$

##### Calculate log likelihood
- Finalmente, iteramos sobre cada palabra en el vocabulario, usaremos la función `lookup` para obtener las frecuencias positivas, $ freq_ {pos} $, y las frecuencias negativas, $ freq_ {neg} $, para esa palabra específica.
- Calcularemos la probabilidad positiva de cada palabra $ P (W_ {pos}) $, la probabilidad negativa de cada palabra $ P (W_ {neg}) $ usando las ecuaciones 4 y 5.

$$ P(W_{pos}) = \frac{freq_{pos} + 1}{N_{pos} + V}\tag{4} $$
$$ P(W_{neg}) = \frac{freq_{neg} + 1}{N_{neg} + V}\tag{5} $$

**Note:** Usaremos un diccionario para almacenar los loglikelihoods de cada palabra. La clave será la palabra, el valor será la probabilidad logarítmica de esa palabra).

- se puede calcular la loglikelihood: $log \left( \frac{P(W_{pos})}{P(W_{neg})} \right)\tag{6}$.

In [8]:
def train_naive_bayes(freqs, train_x, train_y):
    '''
    Input:
        freqs: dictionary from (word, label) to how often the word appears
        train_x: a list of tweets
        train_y: a list of labels correponding to the tweets (0,1)
    Output:
        logprior: the log prior. (equation 3 above)
        loglikelihood: the log likelihood of you Naive bayes equation. (equation 6 above)
    '''
    loglikelihood = {}
    logprior = 0

    # calculate V, the number of unique words in the vocabulary
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    # calculate N_pos and N_neg
    N_pos = N_neg = 0
    for pair in freqs.keys():
        # if the label is positive (greater than zero)
        if pair[1] > 0:

            # Increment the number of positive words by the count for this (word, label) pair
            N_pos += freqs[pair]

        # else, the label is negative
        else:

            # increment the number of negative words by the count for this (word,label) pair
            N_neg += freqs[pair]

    # Calculate D, the number of documents
    D = len(train_y)

    # Calculate D_pos, the number of positive documents (*hint: use sum(<np_array>))
    D_pos = (len(list(filter(lambda x: x > 0, train_y))))

    # Calculate D_neg, the number of negative documents (*hint: compute using D and D_pos)
    D_neg = (len(list(filter(lambda x: x <= 0, train_y))))

    # Calculate logprior
    logprior = np.log(D_pos) - np.log(D_neg)

    # For each word in the vocabulary...
    for word in vocab:
        # get the positive and negative frequency of the word
        freq_pos = lookup(freqs,word,1)
        freq_neg = lookup(freqs,word,0)

        # calculate the probability that each word is positive, and negative
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        # calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos/p_w_neg)


    return logprior, loglikelihood


In [9]:
# You do not have to input any code in this cell, but it is relevant to grading, so please do not change anything
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)
print(logprior)
print(len(loglikelihood))

0.0
9085


**Expected Output**:

0.0

9089

# Probando el modelo de Naive Bayes

Ahora que tenemos `logprior` y` loglikelihood`, podemos probar el modelo de Naive Bayes para predecir algunos tweets.

#### Implementación de  `naive_bayes_predict`

Implementaremos la función `naive_bayes_predict` para hacer predicciones de tweets.

* La función toma como entrada el `tweet`,` logprior`, y `loglikelihood`.
* Devuelve la probabilidad de que el tweet pertenezca a la clase positiva o negativa.
* Para cada tweet, se sumarán las probabilidades de cada palabra en el tweet.
* También se agregará el logprior de esta suma para obtener la predicción de sentimiento del tweet

$$ p = logprior + \sum_i^N (loglikelihood_i)$$

#### Nota

Tenga en cuenta que calculamos el "prior" a partir de los datos de entrenamiento y que los datos de entrenamiento se dividen uniformemente entre etiquetas positivas y negativas (4000 tweets positivos y 4000 negativos). Esto significa que el ratio de positivo a negativo 1, y el logprior es 0.

El valor de 0.0 significa que cuando agregamos el logprior al likelihood, simplemente estamos agregando un valor de cero a la probabilidad logarítmica. Sin embargo, recuerde incluir el logprior, porque siempre que los datos no estén perfectamente equilibrados, el logprior será un valor distinto de cero.


In [10]:

def naive_bayes_predict(tweet, logprior, loglikelihood):
    '''
    Input:
        tweet: a string
        logprior: a number
        loglikelihood: a dictionary of words mapping to numbers
    Output:
        p: the sum of all the logliklihoods of each word in the tweet (if found in the dictionary) + logprior (a number)

    '''
    # process the tweet to get a list of words
    word_l = process_tweet(tweet)

    # initialize probability to zero
    p = 0

    # add the logprior
    p += logprior

    for word in word_l:

        # check if the word exists in the loglikelihood dictionary
        if word in loglikelihood:
            # add the log likelihood of that word to the probability
            p += loglikelihood[word]


    return p


In [11]:

# You do not have to input any code in this cell, but it is relevant to grading, so please do not change anything

# Experiment with your own tweet.
my_tweet = 'She smiled.'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print('The expected output is', p)

The expected output is 1.5737794405738943


**Expected Output**:
- The expected output is around 1.57
- The sentiment is positive.

#### Implementando  test_naive_bayes

* Implementaremos `test_naive_bayes` la función para ver el accuracy de las predicciones. 
* La función toma de `test_x`, `test_y`,el log_prior, y el loglikelihood
* Regresa el ccuracy del modelo
* Primeramente, usaremos la función `naive_bayes_predict` para hacer las predicciones para cada tweet en text_x.

In [12]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    """
    Input:
        test_x: A list of tweets
        test_y: the corresponding labels for the list of tweets
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Output:
        accuracy: (# of tweets classified correctly)/(total # of tweets)
    """
    accuracy = 0  # return this properly

    y_hats = []
    for tweet in test_x:
        # if the prediction is > 0
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i = 0

        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    error = np.mean(np.absolute(y_hats-test_y))

    # Accuracy is 1 minus the error
    accuracy = 1-error

    return accuracy


In [13]:
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

Naive Bayes accuracy = 0.9940


**Expected Accuracy**:

0.9940

In [14]:

# Run this cell to test your function
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    # print( '%s -> %f' % (tweet, naive_bayes_predict(tweet, logprior, loglikelihood)))
    p = naive_bayes_predict(tweet, logprior, loglikelihood)
#     print(f'{tweet} -> {p:.2f} ({p_category})')
    print(f'{tweet} -> {p:.2f}')

I am happy -> 2.15
I am bad -> -1.29
this movie should have been great. -> 2.14
great -> 2.14
great great -> 4.28
great great great -> 6.41
great great great great -> 8.55


**Expected Output**:
- I am happy -> 2.15
- I am bad -> -1.29
- this movie should have been great. -> 2.14
- great -> 2.14
- great great -> 4.28
- great great great -> 6.41
- great great great great -> 8.55

### Predecir un tweet propio

In [15]:
# Feel free to check the sentiment of your own tweet below
my_tweet = 'you are bad :('
naive_bayes_predict(my_tweet, logprior, loglikelihood)

-8.802119484044237

# Filtrado de palabras por ratios positivos y negativos

- Algunas palabras tienen un conteo de etiquetas más positivos que otras y pueden considerarse "más positivas". Asimismo, algunas palabras pueden considerarse más negativas que otras.

- Una forma de definir el nivel de positividad o negatividad, sin calcular la probabilidad logarítmica, es comparar la frecuencia positiva con la negativa de la palabra.

    - Tenga en cuenta que también podemos utilizar los cálculos de probabilidad logarítmica para comparar la positividad o negatividad relativa de las palabras.
    
- Podemos calcular el ratio de las frecuencias positivas y negativas de una palabra.
- Una vez que podamos calcular estos ratios, también podemos filtrar un subconjunto de palabras que tengan una proporción mínima de positividad / negatividad o superior.
- De manera similar, también podemos filtrar un subconjunto de palabras que tienen una proporción máxima de positividad / negatividad o menor (palabras que son menos negativas, o incluso más negativas que un umbral dado).

#### Implementación `get_ratio()`

- Dado el diccionario de palabras `freqs` y una palabra en particular, usaremos` lookup (freqs, word, 1) `para obtener el recuento positivo de la palabra.
- De manera similar, usaremos la función `lookup ()` para obtener el recuento negativo de esa palabra.
- Calcularemos la proporción de positivo dividido por conteos negativos.

$$ ratio = \frac{\text{pos_words} + 1}{\text{neg_words} + 1} $$

Where pos_words and neg_words correspond to the frequency of the words in their respective classes. 
<table>
    <tr>
        <td>
            <b>Words</b>
        </td>
        <td>
        Positive word count
        </td>
         <td>
        Negative Word Count
        </td>
  </tr>
    <tr>
        <td>
        glad
        </td>
         <td>
        41
        </td>
    <td>
        2
        </td>
  </tr>
    <tr>
        <td>
        arriv
        </td>
         <td>
        57
        </td>
    <td>
        4
        </td>
  </tr>
    <tr>
        <td>
        :(
        </td>
         <td>
        1
        </td>
    <td>
        3663
        </td>
  </tr>
    <tr>
        <td>
        :-(
        </td>
         <td>
        0
        </td>
    <td>
        378
        </td>
  </tr>
</table>

In [16]:
# UNQ_C8 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def get_ratio(freqs, word):
    '''
    Input:
        freqs: dictionary containing the words
        word: string to lookup

    Output: a dictionary with keys 'positive', 'negative', and 'ratio'.
        Example: {'positive': 10, 'negative': 20, 'ratio': 0.5}
    '''
    pos_neg_ratio = {'positive': 0, 'negative': 0, 'ratio': 0.0}

    # use lookup() to find positive counts for the word (denoted by the integer 1)
    pos_neg_ratio['positive'] = lookup(freqs,word,1)

    # use lookup() to find negative counts for the word (denoted by integer 0)
    pos_neg_ratio['negative'] = lookup(freqs,word,0)

    # calculate the ratio of positive to negative counts for the word
    pos_neg_ratio['ratio'] = (pos_neg_ratio['positive'] + 1)/(pos_neg_ratio['negative'] + 1)

    return pos_neg_ratio


In [17]:
get_ratio(freqs, 'happi')

{'positive': 161, 'negative': 18, 'ratio': 8.526315789473685}

#### Implementación `get_words_by_threshold(freqs,label,threshold)`

* Si establecemos label en 1, buscaremos todas las palabras cuyo umbral de positivo / negativo sea mayoy o igual al umbral.
* Si establecemos la etiqueta en 0, buscaremos todas las palabras cuyo umbral de positivo / negativo sea menor o igual al umbral.
* Utilizaremos la función `get_ratio ()` para obtener un diccionario que contenga el recuento positivo, el recuento negativo y la proporción de recuentos positivos y negativos.
* Agregaremos un diccionario a una lista, donde la clave es la palabra y el diccionario es el diccionario `pos_neg_ratio` que es devuelto por la función` get_ratio () `.

Un ejemplo de este diccionario sería:
```
{'happi':
    {'positive': 10, 'negative': 20, 'ratio': 0.5}
}
```

In [18]:

def get_words_by_threshold(freqs, label, threshold):
    '''
    Input:
        freqs: dictionary of words
        label: 1 for positive, 0 for negative
        threshold: ratio that will be used as the cutoff for including a word in the returned dictionary
    Output:
        word_set: dictionary containing the word and information on its positive count, negative count, and ratio of positive to negative counts.
        example of a key value pair:
        {'happi':
            {'positive': 10, 'negative': 20, 'ratio': 0.5}
        }
    '''
    word_list = {}

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    for key in freqs.keys():
        word, _ = key

        # get the positive/negative ratio for a word
        pos_neg_ratio = get_ratio(freqs, word)

        # if the label is 1 and the ratio is greater than or equal to the threshold...
        if label == 1 and pos_neg_ratio['ratio'] >= threshold:

            # Add the pos_neg_ratio to the dictionary
            word_list[word] = pos_neg_ratio

        # If the label is 0 and the pos_neg_ratio is less than or equal to the threshold...
        elif label == 0 and pos_neg_ratio['ratio'] <= threshold:

            # Add the pos_neg_ratio to the dictionary
            word_list[word] = pos_neg_ratio

        # otherwise, do not include this word in the list (do nothing)

    return word_list


In [19]:
# Test your function: find negative words at or below a threshold
get_words_by_threshold(freqs, label=0, threshold=0.05)

{':(': {'positive': 1, 'negative': 3663, 'ratio': 0.0005458515283842794},
 ':-(': {'positive': 0, 'negative': 378, 'ratio': 0.002638522427440633},
 'zayniscomingbackonjuli': {'positive': 0, 'negative': 19, 'ratio': 0.05},
 '26': {'positive': 0, 'negative': 20, 'ratio': 0.047619047619047616},
 '>:(': {'positive': 0, 'negative': 43, 'ratio': 0.022727272727272728},
 'lost': {'positive': 0, 'negative': 19, 'ratio': 0.05},
 '♛': {'positive': 0, 'negative': 210, 'ratio': 0.004739336492890996},
 '》': {'positive': 0, 'negative': 210, 'ratio': 0.004739336492890996},
 'beli̇ev': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'wi̇ll': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'justi̇n': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'ｓｅｅ': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'ｍｅ': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776}}

In [20]:
# Test your function; find positive words at or above a threshold
get_words_by_threshold(freqs, label=1, threshold=10)

{'followfriday': {'positive': 23, 'negative': 0, 'ratio': 24.0},
 'commun': {'positive': 27, 'negative': 1, 'ratio': 14.0},
 ':)': {'positive': 2847, 'negative': 2, 'ratio': 949.3333333333334},
 'flipkartfashionfriday': {'positive': 16, 'negative': 0, 'ratio': 17.0},
 ':D': {'positive': 498, 'negative': 0, 'ratio': 499.0},
 ':p': {'positive': 103, 'negative': 0, 'ratio': 104.0},
 'influenc': {'positive': 16, 'negative': 0, 'ratio': 17.0},
 ':-)': {'positive': 543, 'negative': 0, 'ratio': 544.0},
 "here'": {'positive': 20, 'negative': 0, 'ratio': 21.0},
 'youth': {'positive': 14, 'negative': 0, 'ratio': 15.0},
 'bam': {'positive': 44, 'negative': 0, 'ratio': 45.0},
 'warsaw': {'positive': 44, 'negative': 0, 'ratio': 45.0},
 'shout': {'positive': 11, 'negative': 0, 'ratio': 12.0},
 ';)': {'positive': 22, 'negative': 0, 'ratio': 23.0},
 'stat': {'positive': 51, 'negative': 0, 'ratio': 52.0},
 'arriv': {'positive': 57, 'negative': 4, 'ratio': 11.6},
 'via': {'positive': 60, 'negative': 1, 

**Observe la diferencia entre los ratios positivos y negativos. Emojis como :( y palabras como 'me' tienden a tener una connotación negativa. Otras palabras como 'glad', 'comunity' y 'arrives' tienden a encontrarse en los tw-eets positivos.**


# Análisis de Error

En esta parte, veremos algunos tweets que el modelo no clasificó correctamente. ¿Por qué crees que ocurrieron las clasificaciones erróneas? ¿Hubo alguna suposición hecha por el modelo de Naive Bayes?

In [21]:

print('Truth Predicted Tweet')
for x, y in zip(test_x, test_y):
    y_hat = naive_bayes_predict(x, logprior, loglikelihood)
    if y != (np.sign(y_hat) > 0):
        print('%d\t%0.2f\t%s' % (y, np.sign(y_hat) > 0, ' '.join(
            process_tweet(x)).encode('ascii', 'ignore')))

Truth Predicted Tweet
1	0.00	b''
1	0.00	b'truli later move know queen bee upward bound movingonup'
1	0.00	b'new report talk burn calori cold work harder warm feel better weather :p'
1	0.00	b'harri niall 94 harri born ik stupid wanna chang :D'
1	0.00	b''
1	0.00	b''
1	0.00	b'park get sunlight'
1	0.00	b'uff itna miss karhi thi ap :p'
0	1.00	b'hello info possibl interest jonatha close join beti :( great'
0	1.00	b'u prob fun david'
0	1.00	b'pat jay'
0	1.00	b'whatev stil l young >:-('


# Predice con tu propio tweet


In [22]:
# Test with your own tweet - feel free to modify `my_tweet`
my_tweet = 'I am happy because I am learning :)'

p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print(p)

9.57402369584527


Congratulations on completing this assignment. See you next week!